In [1]:
import talib
import numpy as np
import pandas as pd
import random
from collections import deque

In [2]:
file_path = '/Users/mymac/Google_Drive/Forex_Robot/'

In [3]:
# ----------------------------------------------------------------------------------------------------
# Get the data
# ----------------------------------------------------------------------------------------------------
df = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2015-2021.csv')
df.Date = pd.to_datetime(df.Date)
df.reset_index(drop=True, inplace=True)

df_small = pd.read_csv(file_path + 'Oanda_Gbp_Usd_M5_2015-2021.csv')
df_small.Date = pd.to_datetime(df_small.Date)
df_small.reset_index(drop=True, inplace=True)

In [4]:
print(df.head())
print('-' * 75)
print(df.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2015-07-30 07:00:00   1.56035   1.56051  1.55900    1.55910   1.56059   
1 2015-07-30 07:05:00   1.55906   1.55956  1.55904    1.55916   1.55924   
2 2015-07-30 07:10:00   1.55922   1.55988  1.55908    1.55978   1.55940   
3 2015-07-30 07:15:00   1.55970   1.55974  1.55912    1.55916   1.55991   
4 2015-07-30 07:20:00   1.55911   1.55975  1.55911    1.55974   1.55929   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.56076  1.55920    1.55929   1.56047   1.56064  1.55910    1.55920  
1   1.55977  1.55924    1.55935   1.55915   1.55966  1.55914    1.55926  
2   1.56004  1.55930    1.55994   1.55931   1.55996  1.55919    1.55986  
3   1.55991  1.55929    1.55933   1.55980   1.55982  1.55920    1.55924  
4   1.55994  1.55928    1.55993   1.55920   1.55984  1.55920    1.55984  
---------------------------------------------------------------------------
                      Date  B

In [5]:
print(df_small.head())
print('-' * 75)
print(df_small.tail())

                 Date  Bid_Open  Bid_High  Bid_Low  Bid_Close  Ask_Open  \
0 2015-07-30 07:00:00   1.56035   1.56051  1.55900    1.55910   1.56059   
1 2015-07-30 07:05:00   1.55906   1.55956  1.55904    1.55916   1.55924   
2 2015-07-30 07:10:00   1.55922   1.55988  1.55908    1.55978   1.55940   
3 2015-07-30 07:15:00   1.55970   1.55974  1.55912    1.55916   1.55991   
4 2015-07-30 07:20:00   1.55911   1.55975  1.55911    1.55974   1.55929   

   Ask_High  Ask_Low  Ask_Close  Mid_Open  Mid_High  Mid_Low  Mid_Close  
0   1.56076  1.55920    1.55929   1.56047   1.56064  1.55910    1.55920  
1   1.55977  1.55924    1.55935   1.55915   1.55966  1.55914    1.55926  
2   1.56004  1.55930    1.55994   1.55931   1.55996  1.55919    1.55986  
3   1.55991  1.55929    1.55933   1.55980   1.55982  1.55920    1.55924  
4   1.55994  1.55928    1.55993   1.55920   1.55984  1.55920    1.55984  
---------------------------------------------------------------------------
                      Date  B

In [6]:
def add_fractal(df, i, look_back=2):
  if i >= look_back and i < df.shape[0] - look_back:
    lows = []
    highs = []

    for j in range(1, look_back + 1):
      prev_bid_low, prev_bid_high = df.loc[df.index[i - j], ['Mid_Low', 'Mid_High']]
      future_bid_low, future_bid_high = df.loc[df.index[i + j], ['Mid_Low', 'Mid_High']]

      lows.append(float(prev_bid_low))
      lows.append(float(future_bid_low))
      highs.append(float(prev_bid_high))
      highs.append(float(future_bid_high))

    bid_low, bid_high = df.loc[df.index[i], ['Mid_Low', 'Mid_High']]

    if float(bid_low) < min(lows):
      return 1

    elif float(bid_high) > max(highs):
      return 2

    else:
      return 0

  else:
    return np.nan

In [7]:
df['macd'], df['macdsignal'], df['macdhist'] = talib.MACD(df['Mid_Close'])
df['ema200'] = talib.EMA(df['Mid_Close'], timeperiod=200)
df['atr'] = talib.ATR(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=500)
df['adx'] = talib.ADX(df['Mid_High'], df['Mid_Low'], df['Mid_Close'], timeperiod=14)
df['rsi'] = talib.RSI(df['Mid_Close'], timeperiod=20)
df['fractal'] = [add_fractal(df, i) for i in range(df.shape[0])]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

print(df['fractal'].value_counts())

0.0    333500
1.0     56752
2.0     54624
Name: fractal, dtype: int64


In [8]:
value_per_pip = 1.0
max_pips_to_risk = 0.0100
amounts_per_day = [-4, -5, -6]

In [9]:
# ----------------------------------------------------------------------------------------------------
# Simulation code
# ----------------------------------------------------------------------------------------------------
def calculate_day_fees(start_date, end_date):
    curr_fee = np.random.choice(amounts_per_day, p=[0.25, 0.50, 0.25])
    num_days = np.busday_count(start_date.date(), end_date.date())

    return num_days * curr_fee


def run_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff, max_open_trades=1):
    pullback_cushion /= 10000
    fractal_distance /= 10000
    spread_cutoff /= 10000
    bar_length /= 10000
    reward = 0
    n_wins = 0
    n_losses = 0
    win_streak = 0
    loss_streak = 0
    curr_win_streak = 0
    curr_loss_streak = 0
    n_buys = 0
    n_sells = 0
    pips_risked = []
    day_fees = 0
    n_units = 50000 / max_open_trades
    trades = deque(maxlen=max_open_trades)

    def iterate_thru_small_data(trade):
        def get_end_date(j):
          return df_small.loc[df_small.index[j + 1], 'Date'] if j + 1 < len(df_small) else None

        j = df_small[df_small.Date >= trade['start_date']].index[0]

        nonlocal reward
        nonlocal day_fees
        nonlocal n_losses
        nonlocal n_wins
        nonlocal curr_win_streak
        nonlocal curr_loss_streak
        nonlocal win_streak
        nonlocal loss_streak

        while j < len(df_small):
          curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close, curr_date = \
          df_small.loc[df_small.index[j], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High', 'Ask_Low',
                                         'Ask_Close', 'Date']]

          if trade['trade_type'] == 'buy' and curr_bid_low <= trade['stop_loss']:
            trade_amount = (trade['stop_loss'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          # if trade['trade_type'] == 'buy' and curr_bid_high - trade['pips_risked'] > trade['stop_loss']:
          #     trade['stop_loss'] = curr_bid_high - trade['pips_risked']

          if trade['trade_type'] == 'buy' and curr_bid_high >= trade['stop_gain']:
            trade_amount = (trade['stop_gain'] - trade['open_price']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          if trade['trade_type'] == 'sell' and curr_ask_high >= trade['stop_loss']:
            trade_amount = (trade['open_price'] - trade['stop_loss']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          # if trade['trade_type'] == 'sell' and trade['pips_risked'] + curr_ask_low < trade['stop_loss']:
          #   trade['stop_loss'] = trade['pips_risked'] + curr_ask_low

          if trade['trade_type'] == 'sell' and curr_ask_low <= trade['stop_gain']:
            trade_amount = (trade['open_price'] - trade['stop_gain']) * trade['n_units'] * value_per_pip
            reward += trade_amount
            day_fees += calculate_day_fees(trade['start_date'], curr_date)

            n_wins += 1 if trade_amount > 0 else 0
            n_losses += 1 if trade_amount < 0 else 0
            curr_win_streak = 0 if trade_amount < 0 else curr_win_streak + 1
            curr_loss_streak = 0 if trade_amount > 0 else curr_loss_streak + 1

            if curr_win_streak > win_streak:
              win_streak = curr_win_streak

            if curr_loss_streak > loss_streak:
              loss_streak = curr_loss_streak

            trade['end_date'] = get_end_date(j)

            break

          j += 1


    i = 2

    while i < len(df):
        curr_date = df.loc[df.index[i], 'Date']

        # Remove any expired trades -----------------------------------------
        trades_to_remove = []

        for trade in trades:
          if trade['end_date'] is not None and curr_date >= trade['end_date']:
            trades_to_remove.append(trade)

        for trade in trades_to_remove:
          trades.remove(trade)
        # -------------------------------------------------------------------

        ema200 = df.loc[df.index[i - 1], 'ema200']
        atr = df.loc[df.index[i - 1], 'atr']
        adx = df.loc[df.index[i - 1], 'adx']
        rsi = df.loc[df.index[i - 1], 'rsi']
        curr_ao = df.loc[df.index[i], 'Ask_Open']
        curr_bo = df.loc[df.index[i], 'Bid_Open']
        spread = abs(curr_ao - curr_bo)
        enough_volatility = spread <= spread_cutoff
        curr_al = df.loc[df.index[i - 1], 'Ask_Low']
        curr_bh = df.loc[df.index[i - 1], 'Bid_High']
        curr_ml = df.loc[df.index[i - 1], 'Mid_Low']
        curr_mh = df.loc[df.index[i - 1], 'Mid_High']
        curr_bar_length = abs(curr_mh - curr_ml)
        macd2, macdsignal2 = df.loc[df.index[i - 2], ['macd', 'macdsignal']]
        macd1, macdsignal1 = df.loc[df.index[i - 1], ['macd', 'macdsignal']]
        macd_vals = [macd2, macdsignal2, macd1, macdsignal1]
        trending = adx >= adx_cutoff

        process_trade = False

        if macd2 < macdsignal2 and macd1 > macdsignal1 and curr_ml > ema200 and enough_volatility and trending and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_buys = True

            for trade in trades:
                if trade['trade_type'] != 'buy':
                    all_buys = False
                    break

            if all_buys:
              pullback = None
              u = i - 3

              while u >= 0:
                  if df.loc[df.index[u], 'fractal'] == 1:
                      curr_pullback = float(df.loc[df.index[u], 'Bid_Low'])
                      curr_fractal_distance = curr_ao - curr_pullback

                      if curr_fractal_distance >= fractal_distance:
                          pullback = curr_pullback
                          break

                  u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_ask_open)
                  stop_loss = round(pullback - pullback_cushion, 5)
                  # stop_loss = round(pullback - pullback_cushion, 3)

                  if stop_loss < open_price:
                      curr_pips_to_risk = open_price - stop_loss

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price + (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'buy', 'stop_loss': stop_loss,
                                                            'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                                            'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_buys += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        elif macd2 > macdsignal2 and macd1 < macdsignal1 and curr_mh < ema200 and enough_volatility and trending and len(trades) < max_open_trades and curr_bar_length <= bar_length:
            all_sells = True

            for trade in trades:
                if trade['trade_type'] != 'sell':
                    all_sells = False
                    break

            if all_sells:
              pullback = None
              u = i - 3

              while u >= 0:
                  if df.loc[df.index[u], 'fractal'] == 2:
                      curr_pullback = float(df.loc[df.index[u], 'Ask_High'])
                      curr_fractal_distance = curr_pullback - curr_bo

                      if curr_fractal_distance >= fractal_distance:
                          pullback = curr_pullback
                          break

                  u -= 1

              if pullback is not None:
                  curr_bid_open, curr_bid_high, curr_bid_low, curr_bid_close, curr_ask_open, curr_ask_high, curr_ask_low, curr_ask_close = \
                  df.loc[df.index[i], ['Bid_Open', 'Bid_High', 'Bid_Low', 'Bid_Close', 'Ask_Open', 'Ask_High',
                                                'Ask_Low', 'Ask_Close']]
                  open_price = float(curr_bid_open)
                  stop_loss = round(pullback + pullback_cushion, 5)
                  # stop_loss = round(pullback + pullback_cushion, 3)

                  if stop_loss > open_price:
                      curr_pips_to_risk = stop_loss - open_price

                      if curr_pips_to_risk <= max_pips_to_risk:
                          stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 5)
                          # stop_gain = round(open_price - (curr_pips_to_risk * risk_reward_ratio), 3)

                          trades.append({'open_price': open_price, 'trade_type': 'sell', 'stop_loss': stop_loss,
                                  'stop_gain': stop_gain, 'pips_risked': round(curr_pips_to_risk, 5),
                                  'n_units': n_units, 'start_date': curr_date, 'end_date': None})

                          n_sells += 1

                          pips_risked.append(curr_pips_to_risk)

                          process_trade = True

        if process_trade:
          iterate_thru_small_data(trades[-1])

        i += 1

    return reward + day_fees, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak

In [10]:
# ----------------------------------------------------------------------------------------------------
# Run simulation
# ----------------------------------------------------------------------------------------------------
risk_reward_ratio_vals = [1.9, 2]
# risk_reward_ratio_vals = [2]
pullback_cushion_vals = [5, 10, 15]
# pullback_cushion_vals = [15]
fractal_distances = [5, 10, 15, 20]
# fractal_distances = [20]
bar_lengths = [5, 10]
# bar_lengths = [5]
adx_cutoffs = [20, 25, 30]
# adx_cutoffs = [0]
spread_cutoffs = [2, 3, 4]
# spread_cutoffs = [4]
n_possibilities = len(risk_reward_ratio_vals) * len(pullback_cushion_vals) * len(fractal_distances) * len(bar_lengths) * len(adx_cutoffs) * len(spread_cutoffs)
all_combos = []

for risk_reward_ratio in risk_reward_ratio_vals:
  for pullback_cushion in pullback_cushion_vals:
    for fractal_distance in fractal_distances:
      for bar_length in bar_lengths:
        for adx_cutoff in adx_cutoffs:
          for spread_cutoff in spread_cutoffs:
            all_combos.append((risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff))

# percentage_to_try = 0.5
percentage_to_try = 1
n_runs = int(percentage_to_try * len(all_combos))
combos_to_try = random.sample(all_combos, n_runs)
print('Num runs: '+ str(len(combos_to_try)) + '\n')

best_risk_reward = None
best_pullback_cushion = None
best_last_x_bars_val = None
best_fractal_distance = None
best_bar_length = None
best_adx_cutoff = None
best_spread_cutoff = None
top_n_results = 20
best_rewards = []
best_reward = -np.inf
runs_finished = 0

for risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff in combos_to_try:
  reward, n_buys, n_sells, n_wins, n_losses, win_streak, loss_streak = run_simulation(risk_reward_ratio, pullback_cushion, fractal_distance, bar_length, adx_cutoff, spread_cutoff)
  runs_finished += 1

  print(reward)
  print('Num buys: ' + str(n_sells))
  print('Num sells: ' + str(n_buys))
  print('Num trades: ' + str(n_buys + n_sells))
  print('Num wins: ' + str(n_wins))
  print('Num losses: ' + str(n_losses))
  print('Win streak: ' + str(win_streak))
  print('Loss streak: ' + str(loss_streak))
  print('Remaining runs: ' + str(n_runs - runs_finished))

  min_item = min(best_rewards, key=lambda entry: entry['reward']) if len(best_rewards) >= top_n_results else None
  if min_item is None or (reward > 0 and reward > min_item['reward']):
    if min_item is not None:
      best_rewards.remove(min_item)
    best_rewards.append({'reward': int(reward), 'ratio': risk_reward_ratio, 'pullback': pullback_cushion, 'fractal_dist': fractal_distance, 'bar_length': bar_length, 'adx': adx_cutoff, 'spread': spread_cutoff})


  if reward > best_reward:
    best_reward = reward
    best_risk_reward = risk_reward_ratio
    best_pullback_cushion = pullback_cushion
    best_fractal_distance = fractal_distance
    best_bar_length = bar_length
    best_adx_cutoff = adx_cutoff
    best_spread_cutoff = spread_cutoff

  print('Best reward so far: ' + str(best_reward))
  print()

Num runs: 432

-12830.499999999807
Num buys: 472
Num sells: 486
Num trades: 958
Num wins: 306
Num losses: 651
Win streak: 8
Loss streak: 16
Remaining runs: 431
Best reward so far: -12830.499999999807

-10674.9999999999
Num buys: 596
Num sells: 601
Num trades: 1197
Num wins: 400
Num losses: 796
Win streak: 7
Loss streak: 20
Remaining runs: 430
Best reward so far: -10674.9999999999

-9179.500000000022
Num buys: 537
Num sells: 540
Num trades: 1077
Num wins: 350
Num losses: 726
Win streak: 6
Loss streak: 17
Remaining runs: 429
Best reward so far: -9179.500000000022

-9017.999999999494
Num buys: 495
Num sells: 504
Num trades: 999
Num wins: 316
Num losses: 683
Win streak: 6
Loss streak: 13
Remaining runs: 428
Best reward so far: -9017.999999999494

-11838.500000000156
Num buys: 529
Num sells: 521
Num trades: 1050
Num wins: 315
Num losses: 734
Win streak: 5
Loss streak: 21
Remaining runs: 427
Best reward so far: -9017.999999999494

-2493.4999999998468
Num buys: 272
Num sells: 271
Num trades: 

KeyboardInterrupt: 

In [ ]:
print('------------ FINAL RESULTS ------------')
print('Best reward: ' + str(best_reward))
print('Best risk/reward ratio: ' + str(best_risk_reward))
print('Best pullback cushion: ' + str(best_pullback_cushion))
print('Best fractal distance: ' + str(best_fractal_distance))
print('Best bar length: ' + str(best_bar_length))
print('Best spread: ' + str(best_spread_cutoff))
print('Best adx: ' + str(best_adx_cutoff))
print('-----------------------')
print('Top results:')
for entry in best_rewards:
    print(entry)

# 2015-2016:
# {'reward': 430, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 561, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 473, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 633, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 694, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 947, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 603, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 471, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 617, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 340, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 346, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 4}
# {'reward': 484, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 389, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 485, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 458, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 365, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 436, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 598, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 5, 'adx': 25, 'spread': 4}
# {'reward': 1207, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 4}
# {'reward': 755, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}


# 2016-2017:
# {'reward': 282, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 4}
# {'reward': 366, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 1116, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 357, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 409, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 298, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 417, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 952, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 4}
# {'reward': 688, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 4}
# {'reward': 690, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 403, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 414, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 328, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 809, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 463, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 506, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 252, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 279, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 449, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 280, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 2}

# 2017-2018:
# {'reward': 1475, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 1225, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 2453, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1273, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 1397, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 4}
# {'reward': 1412, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1450, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 1240, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 4}
# {'reward': 1494, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 2601, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1375, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 3043, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 2841, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1375, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1407, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1347, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 4}
# {'reward': 1510, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1575, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1234, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 1239, 'ratio': 2, 'pullback': 15, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 3}

# 2018-2019:
# {'reward': 1472, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 5}
# {'reward': 1118, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 1115, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 5}
# {'reward': 1958, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 1107, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 1597, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 1075, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 1575, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 1055, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 1098, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 5}
# {'reward': 1356, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 994, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 5}
# {'reward': 1050, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 1004, 'ratio': 2, 'pullback': 10, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 1104, 'ratio': 1.9, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 3}
# {'reward': 1159, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1289, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 1062, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 1027, 'ratio': 1.9, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': 1944, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}

# 2019-2020:
# {'reward': 1019, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1278, 'ratio': 1.9, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1002, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 5}
# {'reward': 967, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1045, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 5}
# {'reward': 1370, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 4}
# {'reward': 1053, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 992, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 4}
# {'reward': 1264, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 4}
# {'reward': 1259, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 982, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 5}
# {'reward': 1052, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 1003, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 1127, 'ratio': 1.9, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 4}
# {'reward': 1081, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 1151, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 5}
# {'reward': 1230, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 5, 'adx': 20, 'spread': 4}
# {'reward': 1069, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 5}
# {'reward': 1013, 'ratio': 1.9, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 4}
# {'reward': 998, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 3}

# 2020-2021:
# {'reward': 1607, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 1718, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 5}
# {'reward': 1253, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 1799, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 5}
# {'reward': 1723, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 1728, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 1295, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1251, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 1637, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 1792, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 1797, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 1706, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 1698, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 3}
# {'reward': 1801, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 1699, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 5}
# {'reward': 1612, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 5}
# {'reward': 1840, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 1605, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 4}
# {'reward': 1757, 'ratio': 1.9, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 10, 'adx': 25, 'spread': 2}
# {'reward': 1262, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 5}






# {'reward': -983, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': -929, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': -2171, 'ratio': 2, 'pullback': 5, 'fractal_dist': 15, 'bar_length': 10, 'adx': 30, 'spread': 4}
# {'reward': -1824, 'ratio': 2, 'pullback': 5, 'fractal_dist': 5, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 31, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': -1641, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 17, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': -1223, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 571, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 685, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 251, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 231, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 28, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 10, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 856, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 2}
# {'reward': 254, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 258, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 4}
# {'reward': 376, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 1019, 'ratio': 2, 'pullback': 5, 'fractal_dist': 10, 'bar_length': 5, 'adx': 20, 'spread': 3}
# {'reward': 690, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}





# No TL:
# {'reward': 1944, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 4}
# {'reward': 902, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 137, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 1604, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 2057, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 217, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 1027, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 172, 'ratio': 2, 'pullback': 5, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
# {'reward': 303, 'ratio': 2, 'pullback': 10, 'fractal_dist': 5, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 1616, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 1428, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 4}
# {'reward': 2319, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}
# {'reward': 1016, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 3}
# {'reward': 3148, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 882, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 177, 'ratio': 2, 'pullback': 15, 'fractal_dist': 5, 'bar_length': 5, 'adx': 25, 'spread': 2}
# {'reward': 2428, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 69, 'ratio': 2, 'pullback': 15, 'fractal_dist': 15, 'bar_length': 10, 'adx': 20, 'spread': 3}
# {'reward': 875, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
# {'reward': 2783, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 4}


------------ FINAL RESULTS ------------
Best reward: 3148.499999999932
Best risk/reward ratio: 2
Best pullback cushion: 15
Best fractal distance: 20
Best bar length: 10
Best spread: 3
Best adx: 20
-----------------------
Top results:
{'reward': 1944, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 4}
{'reward': 902, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
{'reward': 137, 'ratio': 2, 'pullback': 10, 'fractal_dist': 10, 'bar_length': 5, 'adx': 30, 'spread': 2}
{'reward': 1604, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 2}
{'reward': 2057, 'ratio': 2, 'pullback': 15, 'fractal_dist': 20, 'bar_length': 10, 'adx': 20, 'spread': 2}
{'reward': 217, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 4}
{'reward': 1027, 'ratio': 2, 'pullback': 10, 'fractal_dist': 20, 'bar_length': 5, 'adx': 30, 'spread': 3}
{'reward': 172, 'ratio': 